<a href="https://colab.research.google.com/github/shawrk3387/EVA4_Assignments/blob/master/EVA4_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3)
        self.batch_norm1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 8, 3)
        self.batch_norm2 = nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8, 8, 3)
        self.batch_norm3 = nn.BatchNorm2d(8)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.1)

        self.conv4 = nn.Conv2d(8, 32, 3)
        self.batch_norm4 = nn.BatchNorm2d(32)
        self.conv5 = nn.Conv2d(32, 32, 3)
        self.batch_norm5 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout(0.15)

        self.conv6 = nn.Conv2d(32, 10, 3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.batch_norm1(x)
        x = F.relu(self.conv2(x))
        x = self.batch_norm2(x)
        x = F.relu(self.conv3(x))
        x = self.batch_norm3(x)
        x = self.pool1(x)
        x = self.dropout1(x)

        x = F.relu(self.conv4(x))
        x = self.batch_norm4(x)
        x = F.relu(self.conv5(x))
        x = self.batch_norm5(x)
        x = self.pool2(x)
        x = self.dropout2(x)

        x = self.conv6(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

In [85]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
            Conv2d-3            [-1, 8, 24, 24]             584
       BatchNorm2d-4            [-1, 8, 24, 24]              16
            Conv2d-5            [-1, 8, 22, 22]             584
       BatchNorm2d-6            [-1, 8, 22, 22]              16
         MaxPool2d-7            [-1, 8, 11, 11]               0
           Dropout-8            [-1, 8, 11, 11]               0
            Conv2d-9             [-1, 32, 9, 9]           2,336
      BatchNorm2d-10             [-1, 32, 9, 9]              64
           Conv2d-11             [-1, 32, 7, 7]           9,248
      BatchNorm2d-12             [-1, 32, 7, 7]              64
        MaxPool2d-13             [-1, 32, 3, 3]               0
          Dropout-14             [-1, 3

In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [88]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0503, Accuracy: 9841/10000 (98.4%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0345, Accuracy: 9884/10000 (98.8%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0199, Accuracy: 9941/10000 (99.4%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0189, Accuracy: 9942/10000 (99.4%)



loss=0.005600383039563894 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 38.24it/s]



Test set: Average loss: 0.0195, Accuracy: 9940/10000 (99.4%)

